In [ ]:
#Ellenőrizzuk mi van a taxis adatbázisban
#Exploratory Data Analysis (Feltáró adatelemzés)
#CTRL nyomva tartva -> GoToDefinition
import pandas as pd
import requests
import os
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

pd.set_option('display.max_columns',30)

data_out = r'../data/'


In [ ]:
api_token = os.environ.get('CHICAGO_API_TOKEN')
headers = {"X-App-Token": api_token}

endDate = dt.now().strftime("%Y-%m-%d")
startDate = (dt.now() - relativedelta(months=2)).strftime("%Y-%m-%d")


#startDate = dt(2023,9,8).strftime("%Y-%m-%d")

#url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000&$$app_token={api_token}"
#url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000"

url_old = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$limit=213000000"
url_new = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$limit=213000000"

params = f"$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{endDate}T23:59:59'&$limit=30000" # ? nem kell, ha így adod meg
response_old = requests.get(url_old)
response_new = requests.get(url_new)
data_old = response_old.json()
data_new = response_new.json()
response_old, response_new

In [ ]:
len(data_new)
data_new_df = pd.DataFrame(data_new)
data_old_df = pd.DataFrame(data_old)


In [ ]:
data_new_df.head(5)

In [ ]:
data_old_df.tail(5)

In [ ]:
https://data.cityofchicago.org/resource/ajtu-isnz.json
https://data.cityofchicago.org/resource/wrvz-psew.json

In [ ]:
#Ezzel lehet megnézni, hogy mi a tipusa egy vátozónak, vagy egy obj hova tartozik
#type(response)
data_new_df.info()

In [ ]:
len(data)

In [ ]:
taxi_trips = pd.concat([data_old_df,data_new_df], ignore_index=True)


In [ ]:
taxi_trips.info()
#Itt lehet azokat az oszlopokat kiszűrni, ahol null van.
#Ellenőrizni a duplikációt, mint pl a koordinátáknál

In [ ]:
taxi_trips.describe()
#A mapping táblák létrehozásához segítség. Ha sok kis unique érték van, akkor célszerű mapping táblát csinálni, hogy ne foglaljon annyi helyet.

In [ ]:
taxi_trips[taxi_trips['fare'].isna()].sample(5)
#Itt a nem érvényes adatokat lehet ignorálni.

Transformations : deal with NaN values

In [ ]:
taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
#mert a fele üres

In [ ]:
taxi_trips.info()

In [ ]:
#Koordináták duplikációja miatt
taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)

In [ ]:
taxi_trips.info()

Transformation : renaming community area colunms

In [ ]:
taxi_trips.rename(columns={'pickup_community_area' : 'pickup_community_area_id',
                           'dropoff_community_area' : 'dropoff_community_area_id'
                           },inplace=True)

In [ ]:
taxi_trips.columns

Transformation : Create helper col for weather

In [ ]:
taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp'])
taxi_trips['datetime_for_weather'] = taxi_trips['trip_start_timestamp'].dt.floor('H')

In [ ]:
taxi_trips['datetime_for_weather']

Transformations : check joining trips to weather

In [ ]:
#Get weather data
url = f"https://archive-api.open-meteo.com/v1/era5"
startDate = '2023-09-08'
params = {
    "latitude" : 41.85,
    "longitude" : -87.65,
    "start_date" : startDate,
    "end_date" : startDate,
    "hourly" : "temperature_2m,wind_speed_10m,precipitation,rain"
}


response = requests.get(url, params=params)
weather_data = response.json()

weather_filtered = {
    'datetime': weather_data['hourly']['time'],
    'temperature' : weather_data['hourly']['temperature_2m'],
    'wind' : weather_data['hourly']['wind_speed_10m'],
    'precipitation' : weather_data['hourly']['precipitation'],
    'rain' : weather_data['hourly']['rain']
}
weather_df = pd.DataFrame(weather_filtered)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])




In [ ]:
weather_df.head()

In [ ]:
#Merging
taxi_trips_with_weather = taxi_trips.merge(weather_df, left_on='datetime_for_weather', right_on='datetime')
taxi_trips_with_weather.drop(['datetime_for_weather', 'datetime'], axis=1, inplace=True)
taxi_trips_with_weather.head()

Transformation : data type conversions

In [ ]:
taxi_trips_with_weather.head(1)

In [ ]:
#Az AWS nem a df-ből, hanem a csv-ből fog dolgozni. A memória használatot is lehet a megfelelő típus konvertálásokkal csökkenteni.
taxi_trips_with_weather.info()

In [ ]:
taxi_trips_with_weather.dropna(inplace=True)

data_types = {
    'trip_end_timestamp' : 'datetime64[ns]',
    'trip_seconds': 'int32',
    'trip_miles': 'float',
    'pickup_community_area_id': 'int8',
    'dropoff_community_area_id': 'int8',
    'fare': 'float',
    'tips': 'float',
    'extras': 'float',
    'trip_total': 'float',

}
taxi_trips_with_weather = taxi_trips_with_weather.astype(data_types)

In [ ]:
taxi_trips_with_weather.info()

Memory usage

In [ ]:
taxi_trips.memory_usage(deep=True).sum() / 1024

In [ ]:
taxi_trips_with_weather.memory_usage(deep=True).sum() / 1024

Sanity checks

In [ ]:
#Megkeresi az értéktartományokat, kiugró értékeket (outliers) 

In [ ]:
taxi_trips_with_weather[taxi_trips_with_weather['trip_end_timestamp'] == taxi_trips_with_weather['trip_end_timestamp'].max()]

In [ ]:
taxi_trips_with_weather.nlargest(10, 'fare')

In [ ]:
print(taxi_trips_with_weather.shape)

#### Data Modelling

In [ ]:
#Mapping table create
taxi_trips_with_weather['payment_type'].unique()

In [ ]:
payment_type_master = taxi_trips_with_weather['payment_type'].drop_duplicates().reset_index(drop=True)

In [ ]:
for i in range(1, len(payment_type_master) + 1):
    print(i)

In [ ]:
payment_type_master = pd.DataFrame({
    'payment_type_id': range(1, len(payment_type_master) + 1),
    'payment_type':  payment_type_master
})

In [ ]:
payment_type_master

In [ ]:
taxi_trips['company'].unique()

In [ ]:
company_type_master = taxi_trips_with_weather['company'].drop_duplicates().reset_index(drop=True)
company_type_master = pd.DataFrame({
    'company_id': range(1, len(company_type_master) + 1),
    'company':  company_type_master
})
company_type_master

In [ ]:
taxi_trips_with_weather[['payment_type', 'company']]

In [ ]:
taxi_trips_with_weather.memory_usage(deep=True).sum() / 1024

In [ ]:
taxi_trips_with_weather_id = taxi_trips_with_weather.merge(payment_type_master, on='payment_type')

taxi_trips_with_weather_id = taxi_trips_with_weather_id.merge(company_type_master, on='company')

In [ ]:
taxi_trips_with_weather_id.sample(5)

In [ ]:
taxi_trips_with_weather_id.drop(['payment_type', 'company'], axis=1, inplace=True)

In [ ]:
taxi_trips_with_weather_id.memory_usage(deep=True).sum() / 1024

In [ ]:
payment_type_master.to_csv(data_out+'payment_type_master.csv', index=False)
company_type_master.to_csv(data_out+'company_master.csv', index=False)


#### Checking the modelling

#### Payment

In [ ]:
#Ha új payment type van azt frissíteni kell
payment_type_master = taxi_trips_with_weather['payment_type'].drop_duplicates().reset_index(drop=True)
payment_type_master = pd.DataFrame({
    'payment_type_id': range(1, len(payment_type_master) + 1),
    'payment_type':  payment_type_master
})

In [ ]:
payment_type_master

In [ ]:
#Kézzel megcsináljuk a meglévő típusokra a df-et
#Most azt szimuláljuk, hogy van 2 új típus
new_payment_type_data = [
    {'payment_type': 'Credit Card'},
    {'payment_type': 'X'},
    {'payment_type': 'Y'},
]

#Mapping

new_payment_type_mapping = pd.DataFrame(new_payment_type_data)

In [ ]:
new_payment_type_mapping

In [ ]:
#Megnézzük, hogy a payment type masterben melyik a max index.
payment_type_max_id = payment_type_master['payment_type_id'].max()

In [ ]:
#Hogyan lehet bővíteni a meglévő payment type-okat újakkal?
#Készítünk az indexnek egy sorozatot, ami a meglévő max értéktől indúl
#a new payment type hossza + payment type max id-ig
range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_mapping)+1)

In [ ]:
#A mapping táblában új oszlop az id-nak
new_payment_type_mapping['payment_type_id'] = range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_mapping)+1)

In [ ]:
new_payment_type_mapping

In [ ]:
#Ellenőrtés, hogy van-e az eredetiban
new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])

In [ ]:
#Szűrjük le a mappinget a False-okra
new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]


In [ ]:
new_payment_types = new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]


In [ ]:
if not new_payment_types.empty:
    payment_type_master = pd.concat([payment_type_master, new_payment_types],ignore_index=True )

In [ ]:
payment_type_master

#### Company data

In [ ]:
company_master = taxi_trips_with_weather['company'].drop_duplicates().reset_index(drop=True)
company_master = pd.DataFrame({
    'company_id': range(1, len(company_master) + 1),
    'company':  company_master
})

In [ ]:
company_master.tail()

In [ ]:
company_master.info()

In [ ]:
new_company_data = [
    {'company': 'Petani Cab Corp'},
    {'company': 'X'},
    {'company': 'Y'},
]

#Mapping

new_company_mapping = pd.DataFrame(new_company_data)

In [ ]:
new_company_mapping

In [ ]:
company_max_id = company_master['company_id'].max()
new_company_mapping['company_id'] = range(company_max_id + 1, company_max_id + len(new_company_mapping)+1)

In [ ]:
new_company_mapping

In [ ]:
#Ellenőrtés, hogy van-e az eredetiban
new_company_mapping['company'].isin(company_master['company'])

In [ ]:
#Szűrjük le a mappinget a False-okra
new_company_mapping[~new_company_mapping['company'].isin(company_master['company'])]

In [ ]:
new_company = new_company_mapping[~new_company_mapping['company'].isin(company_master['company'])]

if not new_company.empty:
    company_master = pd.concat([company_master,new_company],ignore_index=True )

In [ ]:
company_master.tail()